# STA 220 Assignment 2

Due __Februrary 9, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on your proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed 
import numpy as np
from scipy.sparse import csr_matrix
import requests_cache
from lxml import html as lx
import json
from urllib.parse import unquote
from scipy.sparse import save_npz, load_npz

__Exercise 1__

We will compute the [PageRank](https://en.wikipedia.org/wiki/PageRank) of the articles of the [Sinhala](https://en.wikipedia.org/wiki/Sinhala_language) wikipedia, which is available at [si.wikipedia.org](https://si.wikipedia.org/wiki/%E0%B6%B8%E0%B7%94%E0%B6%BD%E0%B7%8A_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94%E0%B7%80). Additional information of the Sinhala wiki can be found [here](https://meta.wikimedia.org/wiki/List_of_Wikipedias). 

_Hints: If you don't speak Sinhalese, you might want to learn the wiki logic from the english wikipedia, and translate your findings. Also, caching is highly recommended._

__(a)__ Use the special [AllPages](https://si.wikipedia.org/wiki/%E0%B7%80%E0%B7%92%E0%B7%81%E0%B7%9A%E0%B7%82:%E0%B7%83%E0%B7%92%E0%B6%BA%E0%B7%85%E0%B7%94_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94) page and understand its logic to retrieve the url of all articles in the sinhalese wikipedia. Make sure to skip redirections.

_How many articles are there?_

In [32]:
# Starting URL of the AllPages page on Sinhala Wikipedia
url = "https://si.wikipedia.org/wiki/%E0%B7%80%E0%B7%92%E0%B7%81%E0%B7%9A%E0%B7%82:%E0%B7%83%E0%B7%92%E0%B6%BA%E0%B7%85%E0%B7%94_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94"

# Map to store article URLs
article_urls = {}

# Function to scrape article URLs from a page
def scrape_article_urls(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "lxml")
    article_list = soup.find("div", {"class": "mw-allpages-body"})
    articles = article_list.find_all("a", href=True)
    for article in articles:
        if(article.get('class', [''])[0] == 'mw-redirect'):
            continue
        article_urls["https://si.wikipedia.org" + article["href"]] = True

    return soup

# Initial scraping of the first page
soup = scrape_article_urls(url)
next_page = (soup.find("div", {"class": "mw-allpages-nav"})).a

while True:
    next_page_url = "https://si.wikipedia.org" + next_page["href"]
    soup = scrape_article_urls(next_page_url)
    next_pages = soup.find("div", {"class": "mw-allpages-nav"}).find_all('a')
    if(len(next_pages) < 2):
        break
    else:
        next_page = next_pages[1]

# Print the total number of articles and a sample of article URLs
print("Total number of articles:", len(article_urls))

Total number of articles: 24233


__(b, i)__ Scan all articles in the sinhalese wikipedia and retrieve all links to other articles. Avoid links to special pages, images or the ones that point to another website. Only count the proper article for links that point to a specific section. Use regular expressions to manage these cases. 
__(ii)__ Make sure to match redirections to their correct destiation article. To this end, find how wikipedia treats redirections and retrieve the true article. _(Help: Try searching for 'uc davis' on en.wikipedia.org')_
__(iii)__ Use threading to request all articles and obtain all links to other articles. _(Attention: This takes about thirty minutes!)_


_How many links to other articles are there?_

In [33]:
def scrape_article_links(url):

    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

    try:
        with requests_cache.CachedSession('wiki_cache') as session:
            response = session.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "lxml")

            article_content = soup.find("div", {"class": "mw-parser-output"})
            links = article_content.find_all('a')

            all_links = []

            ## Excluding special pages and including pages that start with wither /wiki/ or https://si.wikipedia.org/
            pattern = re.compile(r'^https?://si\.wikipedia\.org/wiki/[^:]+$|^/wiki/[^:]+$')

            for link in links:
                href = link.get('href', '')
                if(pattern.match(href)):

                    href = href.split('#')[0]

                    if(not href.startswith("http")):
                        new_url = f'https://si.wikipedia.org{href}'
                    else:
                        new_url = href

                    if(new_url == url):
                        continue

                    if(new_url in article_urls):
                        all_links.append(new_url)
                    elif(link.get('class', [''])[0] == 'mw-redirect'): ## mapping redirections to their final destinations
                        redirect_url = session.get(new_url, headers=headers, allow_redirects=True)
                        redirect_content = lx.fromstring(redirect_url.content)
                        actual_link = redirect_content.xpath('//span[@class="mw-redirectedfrom"]/a/@href')
                        if actual_link and actual_link[0] in article_urls:
                                all_links.append(actual_link[0]) 
            
            return all_links
    
    except Exception as e:
        print(f"Error scraping URL {url}: {e}")
        return []

In [34]:

def get_articles_links_with_threading(article_urls):
    links_dict = {}
    with ThreadPoolExecutor(max_workers=8) as executor:
        future_to_url = {executor.submit(scrape_article_links, url): url for url in article_urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                links = future.result()
                links_dict[url] = links
            except Exception as e:
                print(f"Error processing URL {url}: {e}")

    return links_dict

requests_cache.install_cache('wiki_cache') 
links_dict = get_articles_links_with_threading(article_urls)

In [35]:
count = 0

for links in links_dict.values():
    count += len(links)

print("Total number of links:", count)

Total number of links: 333952


In [36]:
with open('links_dict.json', 'w') as json_file:
    json.dump(links_dict, json_file)

In [37]:
with open('links_dict.json', 'r') as json_file:
    links_dict = json.load(json_file)

In [38]:
# Removing duplicates in destination to speed up computation
for url, links in links_dict.items():
    links_dict[url] = set(links)

__(c)__ Compute the transition matrix (see [here](https://en.wikipedia.org/wiki/Google_matrix) and [here](https://www.amsi.org.au/teacher_modules/pdfs/Maths_delivers/Pagerank5.pdf) for step-by-step instructions). Make sure to tread dangling nodes. You may want to use: 
```
from scipy.sparse import csr_matrix
```

In [39]:
def create_transition_matrix(links_dict, link_to_idx):
    
    total_articles = len(link_to_idx)  # Number of nodes
    rows, cols, data = [], [], []

    for source, destinations in links_dict.items():
        
        if len(destinations) == 0: ## if it is a dangling node
            destinations = set(links_dict.keys())

        source_index = link_to_idx[source]
        link_probability = 1.0 / len(destinations)

        for destination in destinations:
            destination_index = link_to_idx[destination]
            data.append(link_probability)
            cols.append(source_index)
            rows.append(destination_index)

    matrix = csr_matrix((data, (rows, cols)), shape=(total_articles, total_articles))

    return matrix

In [41]:
link_to_idx = {link: index for index, link in enumerate(links_dict.keys())}
transition_matrix = create_transition_matrix(links_dict, link_to_idx)

__(d, i)__ Set the damping factor to `0.85` and compute the PageRank for each article, using fourty iterations and starting with a vector with equal entries. __(ii)__ Obtain the top ten articles in terms of PageRank, and, retrieving the articles again, find the correponding English article, if available. 

_Return the corresponding English article titles of the top ten articles from the Sinhalese wikipedia._

In [40]:
def apply_damping_factor(matrix, damping_factor=0.85):

    n = matrix.shape[0]

    data = [(1 - damping_factor) / n] * n
    rows = [i for i in range(n)]
    cols = [i for i in range(n)]

    damping_matrix = csr_matrix((data, (rows, cols)), shape=(n, n))

    matrix = damping_factor * matrix + damping_matrix

    return matrix

In [42]:
transition_matrix_with_damping_factor = apply_damping_factor(transition_matrix)
save_npz('transition_matrix_with_damping_factor.npz', transition_matrix_with_damping_factor)

In [48]:
transition_matrix_with_damping_factor = load_npz('transition_matrix_with_damping_factor.npz')

In [49]:
n = transition_matrix_with_damping_factor.shape[0]  # Number of articles

# PageRank vector
pagerank_vector = np.ones(n) / n  # Equal entries

for _ in range(40):
    pagerank_vector = transition_matrix_with_damping_factor.dot(pagerank_vector)

top_ten_indices = np.argsort(pagerank_vector)[-10:][::-1]  # Indices of top ten articles

In [50]:
def get_english_article(si_article_url):
    
    article_title = si_article_url.split('/')[-1]
    article_title = unquote(article_title)
    
    # Wikipedia API endpoint for language links
    api_url = "https://si.wikipedia.org/w/api.php"
    
    params = {
        'action': 'query',
        'prop': 'langlinks',
        'titles': article_title,
        'lllang': 'en',
        'format': 'json'
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    page_id = next(iter(data['query']['pages']))
    
    if 'langlinks' in data['query']['pages'][page_id]:
        en_title = data['query']['pages'][page_id]['langlinks'][0]['*']
        en_title = en_title.replace(' ', '_')
        return f"https://en.wikipedia.org/wiki/{en_title}"
    else:
        return "No corresponding English article found."


In [51]:
idx_to_link = {idx: link for link, idx in link_to_idx.items()}
top_si_articles = []
corresponding_en_articles = []
for idx in top_ten_indices:
    top_si_articles.append(idx_to_link[idx])

for si_url in top_si_articles:
    corresponding_en_articles.append(get_english_article(si_url))

|English|
|:-|
|https://en.wikipedia.org/wiki/Sri_Lanka|
|https://en.wikipedia.org/wiki/English_language|
|https://en.wikipedia.org/wiki/Gross_domestic_product|
|https://en.wikipedia.org/wiki/United_Kingdom|
|https://en.wikipedia.org/wiki/Geographic_coordinate_system|
|https://en.wikipedia.org/wiki/List_of_decades,_centuries,_and_millennia|
|https://en.wikipedia.org/wiki/Economics|
|https://en.wikipedia.org/wiki/Purchasing_power_parity|
|https://tinyurl.com/englishLanguageAndRace (No corresponding English article found)|
|https://tinyurl.com/centuryList (No corresponding English article found)|

|Sinhala|English|
|:-|:-|
|https://si.wikipedia.org/wiki/ශ්%E2%80%8Dරී_ලංකාව|https://en.wikipedia.org/wiki/Sri_Lanka|
|https://si.wikipedia.org/wiki/ඉංග්%E2%80%8Dරීසි_භාෂාව|https://en.wikipedia.org/wiki/English_language|
|https://si.wikipedia.org/wiki/දළ_දේශීය_නිෂ්පාදිතය|https://en.wikipedia.org/wiki/Gross_domestic_product|
|https://si.wikipedia.org/wiki/එක්සත්_රාජධානිය| https://en.wikipedia.org/wiki/United_Kingdom|
|https://si.wikipedia.org/wiki/භූගෝලීය_ඛණ්ඩාංක_පද්ධතිය| https://en.wikipedia.org/wiki/Geographic_coordinate_system|
|https://si.wikipedia.org/wiki/දශක_ලැයිස්තුව|https://en.wikipedia.org/wiki/List_of_decades,_centuries,_and_millennia|
|https://si.wikipedia.org/wiki/ආර්ථික_විද්%E2%80%8Dයාව | https://en.wikipedia.org/wiki/Economics|
|https://si.wikipedia.org/wiki/ක්%E2%80%8Dරය_ශක්ති_සාම්%E2%80%8Dයය| https://en.wikipedia.org/wiki/Purchasing_power_parity|
|https://si.wikipedia.org/wiki/ඉංග්%E2%80%8Dරීසි| No corresponding English article found.|
|https://si.wikipedia.org/wiki/සියවස්_ලැයිස්තුව|No corresponding English article found.|